# Matrix Factorization
* Prediction is $\tilde R = UA^T$ 
* Loss fuction is $L = \lVert (R - \tilde R)^\Omega \rVert _2^2 + \lambda_u \lVert U \rVert _2^2 + \lambda_a \lVert A \rVert _2^2$
* $\Omega$ is the set of oberved pairs $(i, j)$
* $M^\Omega$ is the projection of $M$ onto $\Omega$ for any matrix $M$
* $U$ is an $m x k$ matrix, $A$ is an $n x k$ matrix and $R$ is the $m x n$ ratings matrix

In [1]:
name = "MatrixFactorization";
residual_alphas = ["UserItemBiases"];

In [2]:
using LinearAlgebra
using SparseArrays

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

# Alternating Least Squares Algorithm
* $u_{ik} = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - \tilde r_{ij} + u_{ik}a_{kj})}{\sum_{j \in \Omega_i} a_j^2 + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function make_prediction(users, items, U, A)
    r = zeros(eltype(U), length(users))
    Threads.@threads for i = 1:length(r)
        if (users[i] <= size(U)[1]) && (items[i] <= size(A)[1])
            r[i] = dot(U[users[i], :], A[items[i], :])
        end
    end
    return r
end;

In [5]:
function calc_loss(df, U, A)
    return mse(df.rating, make_prediction(df.user, df.item, U, A))
end;

In [6]:
function ridge_regression(X, y, λ)
    return (Matrix(X'X) + λ * I(size(X)[2])) \ Vector(X'y)
end;

In [7]:
function sparse_csr(i, j, v, m, n)
    return sparse(j, i, v, n, m)'
end;

In [8]:
function sparse_subset(A, rows)
    # returns a sparse matrix B such that
    # 1) size(B) == size(A)
    # 2) B[rows, :] = A[rows, :]
    # 3) B[i, :] = 0 if i not in rows
    K = size(A)[2]
    nzval = vec(A[rows, :])
    rowval = repeat(rows, K)
    colptr = [1 + (x - 1) * length(rows) for x = 1:K+1]
    return SparseMatrixCSC(size(A)..., colptr, rowval, nzval)
end;

In [9]:
function update_users!(users, items, ratings, U, A, λ_u)
    R = sparse_csr(users, items, ratings, size(U)[1], size(A)[1])
    @tprogress Threads.@threads for i = 1:size(U)[1]
        X = sparse_subset(A, rowvals(R[i, :]))
        y = R[i, :]
        U[i, :] = ridge_regression(X, y, λ_u)
    end
end;

In [10]:
function train_model(training, validation, λ_u, λ_a, K, stop_criteria)
    @debug "training model with parameters [$λ_u, $λ_a]"
    users, items, ratings = training.user, training.item, training.rating
    U = zeros(eltype(λ_u), maximum(users), K) + randn(maximum(users), K)
    A = zeros(eltype(λ_a), maximum(items), K) + randn(maximum(items), K)
    loss = calc_loss(validation, U, A)

    while !stop!(stop_criteria, loss)
        update_users!(users, items, ratings, U, A, λ_u)
        update_users!(items, users, ratings, A, U, λ_a)
        loss = calc_loss(validation, U, A)
        @debug "validation loss $loss"
    end
    return U, A, loss
end;

## Training

In [11]:
# TODO rename calc loss 

In [12]:
function validation_mse(λ, K)
    λ = exp.(λ) # ensure λ is nonnegative
    # stop really early so we can spend more computation exploring the parameter space
    stop_criteria = early_stopper(max_iters = 10, patience = 1, min_rel_improvement = 0.01)
    U, A, loss = train_model(training, validation, λ[1], λ[2], K, stop_criteria)
    return loss
end;

In [13]:
K = 2;

In [ ]:
# Find the best regularization hyperparameters
res = optimize(
    λ -> validation_mse(λ, K),
    fill(-4.0, 2),  # intial guess
    LBFGS(), 
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true, g_tol=1e-4, x_tol=0, f_tol=0),
)
λ = exp.(Optim.minimizer(res));

[ Debug: 20211204 20:39:08 training model with parameters [Dual{ForwardDiff.Tag{var"#12#13", Float64}}(0.01831563888873418,0.01831563888873418,0.0), Dual{ForwardDiff.Tag{var"#12#13", Float64}}(0.01831563888873418,0.0,0.01831563888873418)]


In [ ]:
res

In [ ]:
@info "The optimal [λ_u, λ_a] is $(λ)";

In [ ]:
stopper = early_stopper(max_iters = 100, patience = 2, min_rel_improvement = 0.0001)
U, A, loss = train_model(training, λ..., K, stopper);

## Inference

In [ ]:
model(users, items) = make_prediction(users, items, U, A);

In [ ]:
write_predictions(model);

In [ ]:
write_params(Dict("U" => U, "A" => A, "λ" => λ);